In [484]:
from pydantic import BaseModel, conlist, validator
from typing import List, Dict, Union, Optional
from datetime import date
import re
from datetime import datetime as dt
#from schemas import *
from icecream import ic
import re
from typing import Dict, Union, List
from datetime import datetime
from pydantic import BaseModel
from enum import Enum
from dataclasses import dataclass
import json

In [668]:
class VotingSummary(BaseModel):
    APTA: int = None
    APTS: int = None
    APTT: int = None
    CSEC: Optional[int] = None
    NOMI: int = None
    LEGC: Optional[int] = None  # Only for proportional
    BRAN: int = None
    NULO: int = None
    TOTC: int = None

class Candidate(BaseModel):
    code: str = None
    votes: int = None
 
class Party(BaseModel):
    PART: int|None = None
    LEGP: int|None = None
    TOTP: int|None = None
    candidates: Dict[str, Candidate] = {}

class Position(BaseModel):
    CARG: int = None
    TIPO: int = None
    VERC: int = None
    summary: VotingSummary = VotingSummary()
    party: List[Party] = []

class VotingData(BaseModel):
    IDEL: int = None
    position: List[Position] = []

class SecurityData(BaseModel):
    HASH: str = None
    ASSI: str = None

class Metadata(BaseModel):
    ORIG: str = None
    ORLC: str = None
    PROC: int = None
    DTPL: str = None
    PLEI: int = None
    TURN: int = None
    FASE: str = None
    UNFE: str = None
    MUNI: int = None
    ZONA: int = None
    SECA: int = None
    AGRE: List[float] = []
    IDUE: int = None
    IDCA: int = None
    HIQT: int = None
    HICA: str = None
    VERS: str = None

class Details(BaseModel):
    LOCA: int = None
    APTO: int = None
    APTS: int = None
    APTT: int = None
    COMP: int = None
    FALT: int = None
    HBBM: int = None 
    HBBG: int = None
    HBSB: int = None
    DTAB: str = None
    HRAB: str = None
    DTFC: str = None
    HRFC: str = None

class Header(BaseModel):
    QRBU: list[int] = []
    VRQR: str = None
    VRCH: str = None

class Content(BaseModel):
    metadata: Metadata = Metadata()
    details: Details = Details()
    voting: VotingData = VotingData()
    security: SecurityData = SecurityData()

class BoletimType(Enum):
    SMALL = "small"
    LARGE = "large"

class BoletimUrna(BaseModel):
    type: str
    finished: bool
    last_carg: int
    last_party: int|None
    header: Header
    content: Content
    
class BoletimUrnaContainer(BaseModel):
    bu: List[BoletimUrna]

In [669]:
class ProcessingStep(Enum):
        WAITING: str = "waiting"
        OPEN: str = "open"
        FINISHED: str = "finished"

@dataclass
class ProcessingStatus:
    header_1: ProcessingStep = ProcessingStep.OPEN
    context_setup_1: ProcessingStep = ProcessingStep.WAITING
    content_2: ProcessingStep = ProcessingStep.WAITING
    metadata_2_1: ProcessingStep = ProcessingStep.WAITING
    details_2_2: ProcessingStep = ProcessingStep.WAITING
    voting_2_3: ProcessingStep = ProcessingStep.WAITING
    position_2_3_1: ProcessingStep = ProcessingStep.WAITING
    party_2_3_1_1: ProcessingStep = ProcessingStep.WAITING
    candidate_2_3_1_1_1: ProcessingStep = ProcessingStep.WAITING
    summary_2_3_1_2: ProcessingStep = ProcessingStep.WAITING
    security_2_4: ProcessingStep = ProcessingStep.OPEN

In [711]:
class BulletinUrnaParser:
    def __init__(self, id: int):
        self.id = id
        self.parsing_processing_status = ProcessingStatus()
        self.header = Header()
        self.content = Content()
        self.current_position = None
        self.current_party = None
        self.current_candidates = {}
        self.parsedBulletin = None
        self.party_counter = 0
        self.header_count = [3, 0]
        self.metadata_count = [17, 0]
        self.details_count = [13, 0]
        self.voting_count = [2, 0]
        self.position_count = [4]
        self.counter = 0
    
    def _get_last_position_and_party(self):
        mock_db = {
            1: 'bu_big_1.json'
        }
        bulletin = BoletimUrna(**json.load(open(mock_db[self.id])))
        content = bulletin.content
        return bulletin, content
        
    
    def _parse_header(self, key_value: list):
        if key_value[0] in ["QRBU", "VRQR", "VRCH"] and self.parsing_processing_status.header_1 == ProcessingStep.OPEN:
            print(f"executing header {key_value}")
            if key_value[0] == "QRBU":
                self.header.QRBU = [int(k) for k in key_value[1:]]
                return True
            elif key_value[0] == "VRQR":
                self.header.VRQR = key_value[1]
                return True
            elif key_value[0] == "VRCH":
                self.header.VRCH = key_value[1]
                self.parsing_processing_status.header_1 = ProcessingStep.FINISHED
                self.parsing_processing_status.context_setup_1 = ProcessingStep.OPEN
                return True

    def _is_first_code(self):
        return self.header.QRBU[0] == 1
    
    def _is_unique_code(self):
        return BoletimType.SMALL if self.header.QRBU[1] == 1 else BoletimType.LARGE
    
    def _is_last_code(self):
        return self.header.QRBU[0] == self.header.QRBU[1]

    def _set_context(self):
        if self.parsing_processing_status.context_setup_1 == ProcessingStep.OPEN:
            self.parsing_processing_status.context_setup_1 = ProcessingStep.FINISHED
            self.parsing_processing_status.content_2 = ProcessingStep.OPEN
            if not self._is_first_code():
                self.bulletin, self.content = self._get_last_position_and_party()
                self.bulletin.header.QRBU[0] = self.header.QRBU[0]
                self.parsing_processing_status.metadata_2_1 = ProcessingStep.FINISHED
                self.parsing_processing_status.details_2_2 = ProcessingStep.FINISHED
                self.parsing_processing_status.voting_2_3 = ProcessingStep.OPEN
            else:
                self.parsing_processing_status.metadata_2_1 = ProcessingStep.OPEN 
    
    def _parse_metadata(self, key_value: list):
        if self.parsing_processing_status.metadata_2_1 == ProcessingStep.OPEN and key_value[0] in ["ORIG", "ORLC", "FASE", "UNFE", "HICA", "VERS", "PROC", "PLEI", "TURN", "MUNI", "ZONA", "SECA", "IDUE", "IDCA", "HIQT", "DTPL", "AGRE"]:
            print(f"executing metadata {key_value}")
            if key_value[0] == "ORIG":
                self.content.metadata.ORIG = key_value[1]
                return True
            elif key_value[0] == "ORLC":
                self.content.metadata.ORLC = key_value[1]
                return True
            elif key_value[0] == "FASE":
                self.content.metadata.FASE = key_value[1]
                return True
            elif key_value[0] == "UNFE":
                self.content.metadata.UNFE = key_value[1]
                return True
            elif key_value[0] == "HICA":
                self.content.metadata.HICA = key_value[1]
                return True
            elif key_value[0] == "VERS":
                self.content.metadata.VERS = key_value[1]
                self.parsing_processing_status.metadata_2_1 = ProcessingStep.FINISHED
                self.parsing_processing_status.details_2_2 = ProcessingStep.OPEN
                return True
            elif key_value[0] == "PROC":
                self.content.metadata.PROC = int(key_value[1])
                return True
            elif key_value[0] == "PLEI":
                self.content.metadata.PLEI = int(key_value[1])
                return True
            elif key_value[0] == "TURN":
                self.content.metadata.TURN = int(key_value[1])
                return True
            elif key_value[0] == "MUNI":
                self.content.metadata.MUNI = int(key_value[1])
                return True
            elif key_value[0] == "ZONA":
                self.content.metadata.ZONA = int(key_value[1])
                return True
            elif key_value[0] == "SECA":
                self.content.metadata.SECA = int(key_value[1])
                return True
            elif key_value[0] == "IDUE":
                self.content.metadata.IDUE = int(key_value[1])
                return True
            elif key_value[0] == "IDCA":
                self.content.metadata.IDCA = int(key_value[1])
                return True
            elif key_value[0] == "HIQT":
                self.content.metadata.HIQT = int(key_value[1])
                return True
            elif key_value[0] == "DTPL":
                self.content.metadata.DTPL = datetime.strptime(key_value[1], '%Y%m%d').strftime('%Y-%m-%d')
                return True
            elif key_value[0] == "AGRE":
                self.content.metadata.AGRE = [int(k) for k in key_value[1].split(".")]
                return True

    def _parse_details(self, key_value: list):
        if self.parsing_processing_status.details_2_2 == ProcessingStep.OPEN and key_value[0] in ["LOCA", "APTO", "APTS", "APTT", "COMP", "FALT", "HBBM", "HBBG", "HBSB", "DTAB", "HRAB", "DTFC", "HRFC"]:
            print(f"executing details {key_value}")
            if key_value[0] == "LOCA":
                self.content.details.LOCA = int(key_value[1])
                return True
            elif key_value[0] == "APTO":
                self.content.details.APTO = int(key_value[1])
                return True
            elif key_value[0] == "APTS":
                self.content.details.APTS = int(key_value[1])
                return True
            elif key_value[0] == "APTT":
                self.content.details.APTT = int(key_value[1])
                return True
            elif key_value[0] == "COMP":
                self.content.details.COMP = int(key_value[1])
                return True
            elif key_value[0] == "FALT":
                self.content.details.FALT = int(key_value[1])
                return True
            elif key_value[0] == "HBBM":
                self.content.details.HBBM = int(key_value[1])
                return True
            elif key_value[0] == "HBBG":
                self.content.details.HBBG = int(key_value[1])
                return True
            elif key_value[0] == "HBSB":
                self.content.details.HBSB = int(key_value[1])
                return True
            elif key_value[0] == "DTAB":
                self.content.details.DTAB = datetime.strptime(key_value[1], '%Y%m%d').strftime('%Y-%m-%d')
                return True
            elif key_value[0] == "HRAB":
                self.content.details.HRAB = key_value[1][:2] + ":" + key_value[1][2:4] + ":" + key_value[1][4:]
                return True
            elif key_value[0] == "DTFC":
                self.content.details.DTFC = datetime.strptime(key_value[1], '%Y%m%d').strftime('%Y-%m-%d')
                return True
            elif key_value[0] == "HRFC":
                self.content.details.HRFC = key_value[1][:2] + ":" + key_value[1][2:4] + ":" + key_value[1][4:]
                self.parsing_processing_status.details_2_2 = ProcessingStep.FINISHED
                self.parsing_processing_status.voting_2_3 = ProcessingStep.OPEN
                return True
    
    def _parse_voting(self, key_value: list):
        if key_value[0] in ["IDEL"] and self.parsing_processing_status.voting_2_3 == ProcessingStep.OPEN:
            print(f"executing voting {key_value}")
            self.content.voting.IDEL = int(key_value[1])
            self.parsing_processing_status.position_2_3_1 = ProcessingStep.OPEN
            return True

    def _parse_position(self, key_value: list, parts: list):
        if key_value[0] in ["CARG", "TIPO", "VERC"] and self.parsing_processing_status.position_2_3_1 == ProcessingStep.OPEN:
            print(f"executing position {key_value}")
            if key_value[0] == "CARG":
                self.position_count.append(0)
                if self.current_position:
                    self.content.voting.position.append(self.current_position)
                self.current_position = Position(CARG=int(key_value[1]))
                return True
            elif key_value[0] == "TIPO":
                self.current_position.TIPO = int(key_value[1])
                return True
            elif key_value[0] == "VERC":
                self.current_position.VERC = int(key_value[1])
                self.skip_post_party = False
                self.parsing_processing_status.party_2_3_1 = ProcessingStep.OPEN
                self._verify_party_existence(parts)
                return True
    
    def _verify_party_existence(self, parts: list):
        print(f"verifying party existence {self.next_part.split(":")[0]}")
        if self.next_part.split(":")[0] != "PART":
            print(f"creating empty party")
            self.current_candidates = {}
            self.current_party = Party(PART=None, LEGP=None, TOTP=None)
            self.parsing_processing_status.candidate_2_3_1_1_1 = ProcessingStep.OPEN
            self.skip_post_party = True
    
    def _parse_party(self, key_value: list):
        if key_value[0] in ["PART"] and self.parsing_processing_status.party_2_3_1 == ProcessingStep.OPEN:
            print(f"executing party {key_value}")
            if key_value[0] == "PART":
                self.current_party = None
                self.current_candidates = {}
                self.current_party = Party(PART=int(key_value[1]))
                self.party_counter += 1
                self.parsing_processing_status.candidate_2_3_1_1_1 = ProcessingStep.OPEN
                return True

    def _parse_candidate(self, key_value: list):
        if self.parsing_processing_status.candidate_2_3_1_1_1 == ProcessingStep.OPEN:
            print(f"executing candidate {key_value}")
            try: 
                if int(key_value[0]):
                    print(f"candidate {key_value[0]}")
                    self.current_candidates[key_value[0]] = Candidate(code=key_value[0], votes=int(key_value[1]))
                    print(self.current_candidates)
                    return True
            except:
                print(f"finished by {key_value[0]}")
                self.parsing_processing_status.candidate_2_3_1_1_1 = ProcessingStep.FINISHED
                self.parsing_processing_status.party_2_3_1_1 = ProcessingStep.FINISHED
                if self.skip_post_party:
                    self.parsing_processing_status.summary_2_3_1_2 = ProcessingStep.OPEN
            
    def _parse_post_party(self, key_value: list):
        if key_value[0] in ["LEGP", "TOTP"] and self.parsing_processing_status.party_2_3_1 == ProcessingStep.OPEN:
            print(f"executing post party {key_value}")
            if key_value[0] == "LEGP":
                self.current_party.candidates = self.current_candidates
                self.current_position.party.append(self.current_party)
                self.current_party.LEGP = int(key_value[1])
                return True
            elif key_value[0] == "TOTP":
                self.current_party.TOTP = int(key_value[1])
                self.parsing_processing_status.summary_2_3_1_2 = ProcessingStep.OPEN
                return True
            

    def _parse_summary(self, key_value: list):
        if key_value[0] in ["APTA", 'APTS', "APTT", "NOMI", "BRAN", "NULO", "TOTC", "LEGC"] and self.parsing_processing_status.summary_2_3_1_2 == ProcessingStep.OPEN:
            print(f"executing summary {key_value}")
            if key_value[0] == "APTA":
                self.current_position.summary.APTA = int(key_value[1])
                return True
            elif key_value[0] == "APTS":
                self.current_position.summary.APTS = int(key_value[1])
                return True
            elif key_value[0] == "APTT":
                self.current_position.summary.APTT = int(key_value[1])
                return True
            elif key_value[0] == "NOMI":
                self.current_position.summary.NOMI = int(key_value[1])
                return True
            elif key_value[0] == "BRAN":
                self.current_position.summary.BRAN = int(key_value[1])
                return True
            elif key_value[0] == "NULO":
                self.current_position.summary.NULO = int(key_value[1])
                return True
            elif key_value[0] == "TOTC":
                self.current_position.summary.TOTC = int(key_value[1])
                self.parsing_processing_status.summary_2_3_1_2 = ProcessingStep.FINISHED
                return True
            elif key_value[0] == "LEGC":
                self.current_position.summary.LEGC = int(key_value[1]) if key_value[1] else None
                return True

    def _parse_security(self, key_value: list):
        if key_value[0] in ["HASH", "ASSI"] and self.parsing_processing_status.security_2_4 == ProcessingStep.OPEN:
            print(f"executing security {key_value}")
            if key_value[0] == "HASH":
                if self.current_party:
                    self.current_party.candidates = self.current_candidates
                    self.current_position.party.append(self.current_party)
                    self.last_party = self.current_party.PART
                if self.current_position:
                    self.content.voting.position.append(self.current_position)
                    self.last_carg = self.current_position.CARG
                self.parsing_processing_status.position_2_3_1 = ProcessingStep.FINISHED
                self.parsing_processing_status.voting_2_3 = ProcessingStep.FINISHED
                self.content.security.HASH = key_value[1]
                return True
            elif key_value[0] == "ASSI":
                self.content.security.ASSI = key_value[1]
                self.parsing_processing_status.security_2_4 = ProcessingStep.FINISHED
                self.parsing_processing_status.content_2 = ProcessingStep.FINISHED
                return True

    def execute(self, bu_string: str):
        parts = re.split(r'\s+', bu_string)
        for part in parts:
            key_value = part.split(":")
            print(key_value)
            try:
                self.next_part = parts[self.counter + 1]
                self.counter += 1
            except:
                pass
            if self._parse_header(key_value):
                self.header_count[-1] += 1
                print(f"executed header {self.header_count}, skipping rest \n")
                continue
            self._set_context()

            if self._parse_metadata(key_value):
                self.metadata_count[-1] += 1
                print(f"executed metadata {self.metadata_count}, skipping rest \n")
                continue
            
            if self._parse_details(key_value):
                self.details_count[-1] += 1
                print(f"executed details {self.details_count}, skipping rest \n")
                continue
            
            if self._parse_voting(key_value):
                self.voting_count[-1] += 1
                print(f"executed voting {self.voting_count}, skipping rest \n")
                continue
            
            if self._parse_position(key_value, parts):
                self.position_count[-1] += 1
                print(f"executed position {self.position_count}, skipping rest \n")
                continue
            
            if self._parse_party(key_value):
                print("executed party, skipping rest \n")
                continue
            
            if self._parse_candidate(key_value):
                print("executed candidate, skipping rest \n")
                continue

            if self._parse_post_party(key_value):
                print("executed post party, skipping rest \n")
                continue
            
            if self._parse_summary(key_value):
                print("executed summary, skipping rest \n")
                continue
            
            if self._parse_security(key_value):
                print("executed security, skipping rest \n")
                continue

        self.parsedBulletin = BoletimUrna(
            type=self._is_unique_code().value,
            finished=self._is_last_code(),
            last_carg=self.last_carg,
            last_party=self.last_party,
            header=self.header,
            content=self.content
        )

        return self.parsedBulletin
    
    def export_json(self, name: str):
        with open(name, 'w') as f:
            json.dump(self.parsedBulletin.dict(), f, indent=4)


In [712]:
bu_small_string = {"name": "bu_small.json",
                   "content": "QRBU:1:1 VRQR:1.5 VRCH:20240507 ORIG:VOTA ORLC:LEG PROC:1000 DTPL:20241006 PLEI:1100 TURN:1 FASE:S UNFE:AC MUNI:1120 ZONA:8 SECA:2 AGRE:3.4 IDUE:2031032 IDCA:387626604953598569436326 HIQT:1 HICA:1:387626604953598569436326 VERS:9.20.0.0 LOCA:1 APTO:144 APTS:144 APTT:0 COMP:2 FALT:142 HBBM:0 HBBG:0 HBSB:2 DTAB:20241006 HRAB:172113 DTFC:20241006 HRFC:172300 IDEL:1101 CARG:13 TIPO:1 VERC:202405101700 PART:93 93001:1 LEGP:0 TOTP:1 APTA:144 APTS:144 APTT:0 NOMI:1 LEGC:0 BRAN:1 NULO:0 TOTC:2 CARG:11 TIPO:0 VERC:202405101700 92:1 APTA:144 APTS:144 APTT:0 NOMI:1 BRAN:0 NULO:1 TOTC:2 HASH:57D17C50037E7E4C624468438AE77BEA6562076A20CD454FE30EAD413F7D6174ADE59D0D97013BD8F9F50316D766D3670B57FBB7D396C08DD4C4D9250E7B05FC ASSI:B2FA068D49111BA3A61DA0DC44334F8EC41598C73DE90B8E22AA64DAB8C10AA083FD0737B47560B3C6C837D0F24044ABB18DD5A4D2BC66884DB57BFCFA40F906"}

In [713]:
bu_big_1_string = {"name": "bu_big_1.json",
                   "content": "QRBU:1:4 VRQR:1.5 VRCH:20240507 ORIG:VOTA ORLC:LEG PROC:1000 DTPL:20241006 PLEI:1100 TURN:1 FASE:S UNFE:AC MUNI:1392 ZONA:9 SECA:22 IDUE:2033200 IDCA:216820571350570928893711 HIQT:1 HICA:1:216820571350570928893711 VERS:9.21.0.0 LOCA:4 APTO:559 APTS:559 APTT:0 COMP:504 FALT:55 DTAB:20241006 HRAB:091502 DTFC:20241006 HRFC:170042 IDEL:1101 CARG:13 TIPO:1 VERC:202406131529 PART:91 91001:1 91002:1 91003:4 91004:1 91005:3 91006:3 91007:1 91009:1 91010:1 91011:1 91012:3 91013:1 91014:2 91015:2 91018:1 91020:3 91022:5 91024:2 91025:2 91026:3 91027:3 91028:2 91029:1 91030:2 91031:1 91032:1 91033:2 91034:1 91035:1 91036:1 91037:2 91038:2 91039:5 91040:3 91043:1 91044:4 91045:1 91046:3 91047:2 91048:1 91049:2 91050:1 91051:3 91052:2 91054:2 91055:2 91056:3 91057:1 91059:3 LEGP:0 TOTP:99 PART:92 92001:2 92002:3 92003:2 92004:3 92005:2 92006:2 92007:1 HASH:C84EAF7AEC9D9CD157B5F00206C708B0E6BB67226098903B0050F12D3BEFF79B7EF6025FBA2254E388E264511816BC270EB8701FC9455170FD905BFC6E19628"}

In [714]:
bu_big_2_string = {"name": "bu_big_2.json",
                   "content": "QRBU:2:4 VRQR:1.5 VRCH:20240507 92008:6 92009:2 92010:1 92011:2 92013:1 92014:1 92015:5 92017:3 92018:1 92019:4 92020:3 92021:1 92022:1 92023:1 92024:3 92025:1 92026:2 92027:3 92030:4 92032:2 92033:2 92034:2 92035:1 92036:3 92037:2 92038:1 92039:6 92040:1 92041:1 92042:2 92043:4 92044:2 92047:3 92048:4 92049:1 92050:1 92051:1 92053:2 92054:1 92055:3 92056:1 92057:2 92058:2 92059:2 LEGP:0 TOTP:112 PART:93 93001:2 93002:2 93003:2 93005:2 93006:3 93007:2 93008:5 93009:1 93010:1 93011:2 93012:2 93013:3 93014:1 93015:2 93017:3 93018:1 93019:2 93020:3 93022:1 93023:1 93024:1 93025:5 93026:3 93027:2 93028:1 93029:2 93030:2 93031:1 93032:1 93033:4 93034:2 93035:2 93036:1 93037:1 93039:1 93040:2 93041:1 93042:2 93043:1 93044:1 93045:1 93046:2 93047:1 93048:5 93049:2 93050:1 93052:3 93053:4 93054:2 93055:1 93056:2 93057:2 93058:2 93059:1 LEGP:1 HASH:643D452C83DBADB591895D9D2ACEF3F6E706E3A2E67D53671812BADEF002CC0AE9D16C0DC7BD5F2BF9FDA5F6E616B04974276843375B46D15CF88300FECD7D96"}

In [715]:
bu = bu_big_2_string

In [716]:
parser = BulletinUrnaParser(1)
parser.execute(bu["content"])


['QRBU', '2', '4']
executing header ['QRBU', '2', '4']
executed header [3, 1], skipping rest 

['VRQR', '1.5']
executing header ['VRQR', '1.5']
executed header [3, 2], skipping rest 

['VRCH', '20240507']
executing header ['VRCH', '20240507']
executed header [3, 3], skipping rest 

['92008', '6']


ValidationError: 11 validation errors for BoletimUrna
content.details.HBBM
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type
content.details.HBBG
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type
content.details.HBSB
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type
content.voting.position.0.summary.APTA
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type
content.voting.position.0.summary.APTS
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pyd

In [698]:
# save boletim_urna to json file
parser.export_json(bu["name"])